In [ ]:
Python 元类（Metaclasses）详解
元类（Metaclass）是 Python 中最强大但也最易被误解的高级特性之一。正如 Python 之禅所说：“元类是深度魔法，99% 的用户应该避免使用它。如果你在犹豫是否需要它，那你就不用它。” 但理解元类对掌握 Python 对象模型至关重要。

一、核心概念：一切皆对象，类也是对象
1. 类是对象
在 Python 中，类本身也是对象，是 type 的实例：

In [1]:
class MyClass:
    pass

In [2]:

print(type(MyClass))  # <class 'type'>
print(isinstance(MyClass, type))  # True


<class 'type'>
True


In [ ]:
2. 创建类的两种方式
普通方式（语法糖）： as above


In [ ]:
手动调用 type：

In [3]:

MyClass = type('MyClass', (), {'x': 1})


In [ ]:
✅ type 就是最基础的元类，负责创建所有类。 

二、什么是元类？
元类是“类的类”（class of a class）—— 它定义了类的行为，就像类定义了实例的行为。 

普通对象 ← 由 类 创建
类 ← 由 元类 创建

三、自定义元类的两种方式
方式 1：继承 type（推荐）

In [ ]:
class MyMeta(type):      # <<<<====================================================== use keyword 'class' to define a metaclass
    def __new__(cls, name, bases, attrs):
        # 在类创建前修改 attrs
        attrs['added_by_meta'] = True
        return super().__new__(cls, name, bases, attrs)
    
    def __init__(cls, name, bases, attrs):
        # 在类创建后初始化
        print(f"Initializing class {name}")
        super().__init__(name, bases, attrs)

class MyClass(metaclass=MyMeta):
    pass

print(MyClass.added_by_meta)  # True
# 输出: Initializing class MyClass

Initializing class MyClass
True


In [ ]:
方式 2：函数式元类（简单场景）

In [ ]:
def my_meta(name, bases, attrs):         # use keyword 'def' to define a metaClass <<<<<======================================================
    attrs['added_by_func'] = True
    return type(name, bases, attrs)

class MyClass(metaclass=my_meta):
    pass

print(MyClass.added_by_func)  # True


True


In [ ]:
 #四、元类常用方法 through call


In [6]:

class SingletonMeta(type):
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class Database(metaclass=SingletonMeta):
    pass

db1 = Database()
db2 = Database()
print(db1 is db2)  # True


True


In [ ]:
五、元类 vs 类装饰器


In [ ]:
def singleton(cls):
    instances = {}
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance

@singleton
class Database:
    pass


In [ ]:
class ModelBase(type):
    def __new__(cls, name, bases, attrs):
        # 自动注册模型到 apps
        new_class = super().__new__(cls, name, bases, attrs)
        if name != 'Model':
            apps.register_model(new_class)
        return new_class

class Model(metaclass=ModelBase):
    pass


In [ ]:
2. Pydantic（数据验证）
python



In [ ]:

class ModelMetaclass(type):
    def __new__(cls, name, bases, namespace, **kwargs):
        # 解析类型注解，生成验证器
        namespace['__validators__'] = build_validators(namespace)
        return super().__new__(cls, name, bases, namespace)


In [ ]:

from abc import ABCMeta, abstractmethod

class MyABC(metaclass=ABCMeta):
    @abstractmethod
    def my_method(self):
        pass

class Concrete(MyABC):
    def my_method(self):  # 必须实现
        return "done"


七、执行顺序（关键！）
当定义 class MyClass(metaclass=MyMeta) 时：

MyMeta.__new__ 被调用 → 创建类对象
MyMeta.__init__ 被调用 → 初始化类对象
当调用 MyClass() 时：
MyMeta.__call__ 被调用
→ MyClass.__new__ 创建实例
→ MyClass.__init__ 初始化实例
八、常见误区
❌ 误区 1：元类用于修改实例行为
正解：元类修改类的行为，实例行为由类控制。
❌ 误区 2：__metaclass__ 在 Python 3 有效
正解：Python 3 使用 metaclass=...，__metaclass__ 仅 Python 2 有效。
❌ 误区 3：元类能替代继承
正解：元类用于创建类时的元操作，继承用于代码复用。
✅ 总结：何时使用元类？
需要自动修改类属性
✅ 是
需要注册所有子类
✅ 是
实现单例/ORM/验证框架
✅ 是
简单包装类（如计时）
❌ 用装饰器
修改实例行为
❌ 用
__init__
/
__new__

📌 记住：
“元类是 99% 场景不需要的深度魔法” —— 但当你需要时，它是唯一解。 